# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [25]:
## package imports

import pandas as pd

In [26]:
## reading in json files

"""
Description of json files
* [train-claims,dev-claims].json: JSON files for the labelled training and development set; 
* evidence.json: JSON file containing a large number of evidence passages (i.e. the “knowledge source”); 
* dev-claims-baseline.json: JSON file containing predictions of a baseline system on the development set;
"""

## relative file paths

## baseline system - will not be used for any training/evaluation
devClaimsBaselineFile='./data/dev-claims-baseline.json'
## use this for model training
trainClaimsFile='./data/train-claims.json'
## use this set for hyperparameter tuning and evaluation metric 
devClaimsFile='./data/dev-claims.json'
## evidence files need to be downloaded through https://drive.google.com/file/d/1JlUzRufknsHzKzvrEjgw8D3n_IRpjzo6/view?usp=sharing as it is to big to be uploaded to github
evidenceFile='./data/evidence.json'

## import as pandas dataframe
devClaimsBaseline=pd.read_json(devClaimsBaselineFile)
trainClaims=pd.read_json(trainClaimsFile)
devClaims=pd.read_json(devClaimsFile)
evidence=pd.read_json(evidenceFile,orient='index')

## Separate claim_text,claim_label, and evidences from training and development sets, saved as pd dataframes
claimTextTrain=trainClaims.loc['claim_text'].to_frame()
claimLabelTrain=trainClaims.loc['claim_label'].to_frame()
evidenceTrain=trainClaims.loc['evidences'].to_frame()

claimTextDev=devClaims.loc['claim_text'].to_frame()
claimLabelDev=devClaims.loc['claim_label'].to_frame()
evidenceDev=devClaims.loc['evidences'].to_frame()


In [29]:
## Obtain the evidence texts and add it as a new column at the evidence pd dataframes
## This is in place transformation, but should always overwrite the evidence_text column so does not matter as much

def getTextEvidence(evidenceList):
    texts=[]
    for evidenceID in evidenceList:
        text=evidence.loc[evidenceID]
        texts.append(text)
    return texts

evidenceDev['evidence_text']=evidenceDev['evidences'].apply(getTextEvidence)
evidenceTrain['evidence_text']=evidenceTrain['evidences'].apply(getTextEvidence)

In [30]:
## Preprocessing data 


                                                    evidences  \
claim-752                   [evidence-67732, evidence-572512]   
claim-375   [evidence-996421, evidence-1080858, evidence-2...   
claim-1266                 [evidence-889933, evidence-694262]   
claim-871   [evidence-422399, evidence-702226, evidence-28...   
claim-2164  [evidence-52981, evidence-264761, evidence-947...   
...                                                       ...   
claim-2400  [evidence-409365, evidence-127519, evidence-85...   
claim-204   [evidence-368192, evidence-261690, evidence-20...   
claim-1426  [evidence-1124018, evidence-995813, evidence-1...   
claim-698                                   [evidence-660755]   
claim-1021                [evidence-242575, evidence-1175280]   

                                                evidence_text  
claim-752   [[[citation needed] South Australia has the hi...  
claim-375   [[The 2011 UNEP Green Economy report states th...  
claim-1266  [[Multiple inde

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*